In [1]:
print('helloworld')


helloworld


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import CTransformers
import chromadb
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

extracted_data = load_pdf('data/')

In [4]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

text_chunks = text_split(extracted_data)
print('Length of text chunks:', len(text_chunks))

Length of text chunks: 7020


In [5]:
# Initialize HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings()

embeddings = download_hugging_face_embeddings()
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
query_result

c:\Users\Littleturtle\miniconda3\envs\mchatbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Littleturtle\miniconda3\envs\mchatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Length 768


[0.026249665766954422,
 0.01339563075453043,
 -0.004533134400844574,
 -0.021791353821754456,
 0.054551854729652405,
 -0.004966481123119593,
 0.006655591074377298,
 0.030626269057393074,
 -0.005762884393334389,
 -0.00456205615773797,
 -0.003313267370685935,
 -0.04849628731608391,
 -0.011364093981683254,
 0.03507749363780022,
 0.09309467673301697,
 -0.08668732643127441,
 0.05108647793531418,
 0.009886148385703564,
 -0.0635693371295929,
 -0.008550170809030533,
 0.007054384797811508,
 -0.0038623760920017958,
 0.02474433183670044,
 0.04288493096828461,
 0.03509416803717613,
 -0.029848221689462662,
 0.010252597741782665,
 0.02234489843249321,
 0.02088996209204197,
 0.00949225015938282,
 -0.03304436802864075,
 -0.012284093536436558,
 0.053528930991888046,
 0.02542918734252453,
 2.0221771137585165e-06,
 -0.03419102355837822,
 0.009609990753233433,
 -0.01648455113172531,
 0.005609517451375723,
 -0.004250082653015852,
 -0.022801237180829048,
 0.04035474359989166,
 0.0030519841238856316,
 0.03137

In [6]:
# Initialize ChromaDB
client = chromadb.Client()
collection_name = "soraka-healthcare"
collection = client.create_collection(name=collection_name)

In [ ]:
# Create embeddings for each text chunk and store them
texts = [t.page_content for t in text_chunks]
embeddings_list = [embeddings.embed_query(text) for text in texts]

In [ ]:
# Upsert vectors into ChromaDB
collection.upsert(ids=ids, embeddings=embeddings_list, metadatas=metadatas)

In [ ]:
# Initialize PromptTemplate and RetrievalQA chain
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512,
                            'temperature': 0.8})

In [ ]:
# Create a retriever for ChromaDB
retriever = collection.as_retriever(search_kwargs={'k': 2})

In [ ]:
# Create RetrievalQA instance
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# Interactive QA loop
while True:
    user_input = input("Input Prompt: ")
    result = qa({"query": user_input})
    print("Response : ", result["result"])

Length of text chunks: 7020


c:\Users\Littleturtle\miniconda3\envs\mchatbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Littleturtle\miniconda3\envs\mchatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Length 768


TypeError: Collection.upsert() got an unexpected keyword argument 'vectors'